### 一元配置分散分析 (One-way ANOVA analysis)

**問題**  
plant_growth.csvは異なる3つの条件のもとで育った植物の収穫量のデータである．
３つの条件下での収穫量の母平均に差があるかを有意水準５％で検定する．

**Problem**
plant_growth.csv is the yield data of plants grown under three different conditions.
Test whether there is a difference in the population mean of the yield under the three conditions at the significance level of 5%.

帰無仮説: ３つの条件下での収穫量の母平均に差はない  
対立仮説: ３つの条件下での収穫量の母平均に差がある

Null hypothesis: There is no difference in the population mean of yields under the three conditions  
Alternative hypothesis: There is a difference in the population mean of yields under three conditions

In [1]:
import pandas as pd
import scipy.stats as ss 
import statsmodels.api as sm 
import statsmodels.formula.api as smf

##### csvファイルの読み込み Load data into a pandas's dataframe

In [3]:
datafile = "data/plant_growth.csv"
df = pd.read_csv(datafile, delimiter=",", skiprows=4)
display(df)

,No.,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl
2,3,5.18,ctrl
3,4,6.11,ctrl
4,5,4.50,ctrl
5,6,4.61,ctrl
6,7,5.17,ctrl
7,8,4.53,ctrl
8,9,5.33,ctrl
9,10,5.14,ctrl


##### group列のユニークな要素の値を抽出する Find unique group names

In [4]:
groups = pd.unique(df["group"])
print(groups)

['ctrl' 'trt1' 'trt2']


##### 各グループのデータの抽出　Extraction of data for each group

In [5]:
data1 = df[df["group"]=="ctrl"]["weight"]
data2 = df[df["group"]=="trt1"]["weight"]
data3 = df[df["group"]=="trt2"]["weight"]
print(data1)
#print(data2)
#print(data3)

0    4.17
1    5.58
2    5.18
3    6.11
4    4.50
5    4.61
6    5.17
7    4.53
8    5.33
9    5.14
Name: weight, dtype: float64


### 方法1: 分散分析表から求める方法  How to obtain from the analysis of variance table


分散分析表
<table border="" cellpadding="5" width="100%">
<tbody>
<tr valign="top">
<td align="left" width="8%"><strong>Source</strong></td>
<td align="center" width="22%"><strong>SS</strong></td>
<td align="center" width="19%"><strong>df</strong></td>
<td align="center" width="23%"><strong>MS</strong></td>
<td align="center" width="25%"><strong>F</strong></td>
</tr>
<tr valign="top">
<td align="center"><strong>Between</strong></td>
<td align="left">$SS_{between} = \sum_{j=1}^{k}n_j(\bar{x}_{j} - \bar{x})^2$</td>
<td align="left">$df_{between} = k - 1$</td>
<td align="left">$MSG = \frac{SS_{between}}{df_{between}}$</td>
<td align="left">$ F = \frac{MSG}{MSE} $</td>
</tr>
<tr valign="top">
<td align="center"><strong>Within</strong></td>
<td align="left">$SS_{within} = \sum_{i=1}^{n_j}\sum_{j=1}^{k}(x_{ij} - \bar{x}_{j})^2$</td>
<td align="left">$df_{within} = N - k$</td>
<td align="left">$MSE = \frac{SS_{within}}{df_{within}}$</td>
<td align="center"></td>
</tr>
<tr valign="top">
<td align="center"><strong>Total</strong></td>
<td align="left">$SS_{total} = \sum_{i=1}^{n_j}\sum_{j=1}^{k}(x_{ij} - \bar{x})^2$</td>
<td align="left">$df_{total} = N - 1$</td>
<td align="center"></td>
<td align="center"></td>
</tr>
</tbody></table>


##### 自由度 Degree of freedom

In [6]:
# 水準数 number of levels
k = len(pd.unique(df["group"])) 
# データの総数　total numbers of data
N = df.shape[0]
# degree of freedom between
df_between = k - 1
# degree of freedom within
df_within = N - k
# degree of freedom in total
df_total = N - 1

##### 水準間の平方和 Sum of squares between levels

In [7]:
ave_all = df['weight'].mean() # x_bar: 全てのweightの平均, means of all weights

n1 = len(data1) # "ctrl"のサンプル数 number of samples of "ctrl"
n2 = len(data2) # "trt1"のサンプル数 number of samples of "trt1"
n3 = len(data3) # "trt2"のサンプル数 number of samples of "trt2"
ctrl_ave = data1.mean() # "ctrl"の平均値 mean of "ctrl"
trt1_ave = data2.mean() # "trt1"の平均値 mean of "trt1"
trt2_ave = data3.mean() # "trt2"の平均値 mean of "trt2"

# sum of sum of squares between groups: ss_between
ctrl_ssb = n1 * (ctrl_ave - ave_all)**2
trt1_ssb = n2 * (trt1_ave - ave_all)**2 
trt2_ssb = n3 * (trt2_ave - ave_all)**2
ss_between = ctrl_ssb + trt1_ssb + trt2_ssb
print("ss_between: ", ss_between)

ss_between:  2.792128205128212


##### 水準内の平方和 Sum of squares within levels

In [8]:
# sum of sum of squares within levels: ss_within
# SS_within:
ctrl_ssw = sum((data1 - ctrl_ave)**2)
trt1_ssw = sum((data2 - trt1_ave)**2)
trt2_ssw = sum((data3 - trt2_ave)**2)
ss_within =  ctrl_ssw + trt1_ssw + trt2_ssw
print("ss_within: ", ss_within)

ss_within:  9.589533333333335


##### 総平方和  Sum of squares for total

In [9]:
# sum of sum of squares for total: ss_total
ctrl_sst = sum((data1 - ave_all)**2)
trt1_sst = sum((data2 - ave_all)**2)
trt2_sst = sum((data3 - ave_all)**2)
ss_total = ctrl_sst + trt1_sst + trt2_sst
print("ss_total: ", ss_total)

ss_total:  12.38166153846154


##### F検定  F-statistic

In [10]:
mean_squared_between = ss_between / df_between
mean_squared_within = ss_within / df_within

# F値 F-value
f_ratio =  mean_squared_between / mean_squared_within

# 棄却域(F分布の上側5%点) rejection region (the upper 5-percentile of F-distribution)
print(ss.f.ppf(0.95, df_between, df_within))

# p値 p-value
p_val =  1 - ss.f.cdf(f_ratio, df_between, df_within)

print("F-value: {:.5f}".format(f_ratio))
print("p-value: {:.5f}".format(p_val))

if p_val < 0.05:
    print("Reject H0 (帰無仮説を棄却)")
else:
    print("Retain H0 (帰無仮説を棄却できない)")

3.422132207861178
F-value: 3.34839
p-value: 0.05293
Retain H0 (帰無仮説を棄却できない)


### 方法2: statsmodelsを用いた方法 Method using stats models

Step 1. statsmodels.formula.apiを用いて回帰分析を行う  Construct a regression model using statsmodels.formula.api  
Step 2. 回帰分析の結果に基づいて一元配置分散分析を行う  Perform one-way ANOVA based on the results of regression analysis

In [11]:
model = smf.ols(formula='weight ~ C(group)', data=df) # 回帰モデルの設定  Construct a regression model 
results = model.fit() # 回帰分析の実行 Execute linear regression
print(results.summary()) # 回帰分析の結果 Results of the linear regression

# one-way anova by anova_lm()
table = sm.stats.anova_lm(results, typ=2)
print("----------------")
display(table)
print("----------------")

f = table.at['C(group)','F']
p_val = table.at['C(group)','PR(>F)']
print("F-value: {:.5f}".format(f))
print("p-value: {:.5f}".format(p_val))

if p_val < 0.05:
    print("Reject H0 (帰無仮説を棄却)")
else:
    print("Retain H0 (帰無仮説を棄却できない)")

                            OLS Regression Results                            
Dep. Variable:                 weight   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     3.348
Date:                Sat, 06 Jul 2024   Prob (F-statistic):             0.0529
Time:                        15:13:19   Log-Likelihood:                -23.926
No. Observations:                  26   AIC:                             53.85
Df Residuals:                      23   BIC:                             57.63
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            5.0320      0.204  

,sum_sq,df,F,PR(>F)
C(group),2.792128,2.0,3.348388,0.052931
Residual,9.589533,23.0,NaN,NaN


----------------
F-value: 3.34839
p-value: 0.05293
Retain H0 (帰無仮説を棄却できない)


### 方法3: scipyのf_oneway()関数 scipy's f_oneway() function

In [12]:
f, p_val = ss.f_oneway(data1, data2, data3)
print("F-value: {:.5f}".format(f))
print("p-value: {:.5f}".format(p_val))

if p_val < 0.05:
    print("Reject H0 (帰無仮説を棄却)")
else:
    print("Retain H0 (帰無仮説を棄却できない)")

F-value: 3.34839
p-value: 0.05293
Retain H0 (帰無仮説を棄却できない)


### 解釈 Interpretation

p値が有意水準0.05よりも大きいため，帰無仮説は棄却されない．
つまり，ctrl、trt1、trt2の母平均に差があるとはいえない．

The null hypothesis is not rejected because the p-value is greater than the significance level of 0.05.
Therefore, it cannot be said that there is a difference in the population means of ctrl, trt1 and trt2.

#### 効果量(Effective size)

In [13]:
# eta_squared (somewhat biased)
eta_squared = ss_between / ss_total
# omega_squared
omega_squared = (ss_between - (df_between * mean_squared_within)) / (ss_total + mean_squared_within)
print("eta squared: {:.5f}".format(eta_squared))
print("omega squared: {:.5f}".format(omega_squared))

eta squared: 0.22551
omega squared: 0.15301
